In [8]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [9]:
# parameter 설정

lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './drive/My Drive/Colab Notebooks/MNIST_Test/checkpoint'     # 체크 포인트. 주기마다 모델 저장
log_dir = './drive/My Drive/Colab Notebooks/MNIST_Test/log'             # 텐서 보드 확인을 위한 로그 저장

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # CUDA 디바이스 설정


In [10]:
# Networt 구축

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)      # 출력부 10으로 데이터 확인

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x

In [11]:
# 네트워크 저장 함수

def save(ckpt_dir, net, optim, epoch):        # check point 저장
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)                 # 없다면 체크 포인트 생성

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},    # net, optim -> pth로 저장
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))              # net : , optim : 

def load(ckpt_dir, net, optim):               # check point 로드
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()                           # check point list 한 후 마지막 사용을 위해 sort -> 1, 10도 sort가 되는지 확인

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))         # 마지막 sort를 사용. dict_model에 넣어준다

    net.load_state_dict(dict_model['net'])                                # net, optim 모델에서 로드
    optim.load_state_dict(dict_model['optim'])

    return net, optim

In [12]:
# MNIST 데이터 불러오기

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])      # 텐서 바꾸기 + Normalization = transform

dataset = datasets.MNIST(download=True, root='./drive/My Drive/Colab Notebooks/MNIST_Test', train=False, transform=transform)  # False 변경
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)                             # False 변경

num_data = len(loader.dataset)                      # dataset 개수 확인
num_batch = np.ceil(num_data / batch_size)          # dataset 개수에서 batch_size를 나눠 training batch 개수 구함

In [13]:
# 네트워크 설정, 손실함수 구현

net = Net().to(device)                              # 앞서 설정한 Net 구조 사용 변수
params = net.parameters()                           # Net의 변수 저장

fn_loss = nn.CrossEntropyLoss().to(device)          # loss 함수. CrossEntropyLoss 사용
fn_pred = lambda output: torch.softmax(output, dim=1)         # softmax 사용 predict 모델 생성 변수
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()           # 예측 모델과 실제 모델을 합쳐 정확성 체크

optim = torch.optim.Adam(params, lr=lr)             # Adam 사용. Adam 논문 정독 필요

writer = SummaryWriter(log_dir=log_dir)             # log 저장

# 네트워크 로드 부분 작성

net, optim = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

In [14]:
# 트레이닝

# for epoch in range(1, num_epoch + 1):               # epoch 제거
with torch.no_grad():
    # net.train()
    net.eval()                                      # eval로 변경

    loss_arr = []                                   # Loss 함수 저장
    acc_arr = []                                    # 정확도 함수 저장

    for  batch, (input, label) in enumerate(loader, 1):     # batch 마다 진행
        input = input.to(device)
        label = label.to(device)

        output = net(input)                         # Net에 input
        pred = fn_pred(output)                      # 구한 output softmax

        # optim.zero_grad()                           # Eval이므로 없앰

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        # loss.backward()                             # Eval이므로 없앰

        # optim.step()                                # Eval이므로 없앰

        loss_arr += [loss.item()]                   # Tensorboard 확인을 위한 loss 저장
        acc_arr += [acc.item()]                     # Tensorboard 확인을 위한 acc 저장

        print('TRAIN: BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

TRAIN: BATCH 0001/0157 | LOSS: 0.0023 | ACC 1.0000
TRAIN: BATCH 0002/0157 | LOSS: 0.0053 | ACC 1.0000
TRAIN: BATCH 0003/0157 | LOSS: 0.0085 | ACC 1.0000
TRAIN: BATCH 0004/0157 | LOSS: 0.0218 | ACC 0.9961
TRAIN: BATCH 0005/0157 | LOSS: 0.0219 | ACC 0.9969
TRAIN: BATCH 0006/0157 | LOSS: 0.0281 | ACC 0.9922
TRAIN: BATCH 0007/0157 | LOSS: 0.0347 | ACC 0.9888
TRAIN: BATCH 0008/0157 | LOSS: 0.0386 | ACC 0.9883
TRAIN: BATCH 0009/0157 | LOSS: 0.0396 | ACC 0.9878
TRAIN: BATCH 0010/0157 | LOSS: 0.0456 | ACC 0.9875
TRAIN: BATCH 0011/0157 | LOSS: 0.0479 | ACC 0.9858
TRAIN: BATCH 0012/0157 | LOSS: 0.0497 | ACC 0.9844
TRAIN: BATCH 0013/0157 | LOSS: 0.0464 | ACC 0.9856
TRAIN: BATCH 0014/0157 | LOSS: 0.0441 | ACC 0.9866
TRAIN: BATCH 0015/0157 | LOSS: 0.0478 | ACC 0.9865
TRAIN: BATCH 0016/0157 | LOSS: 0.0499 | ACC 0.9854
TRAIN: BATCH 0017/0157 | LOSS: 0.0507 | ACC 0.9844
TRAIN: BATCH 0018/0157 | LOSS: 0.0506 | ACC 0.9844
TRAIN: BATCH 0019/0157 | LOSS: 0.0492 | ACC 0.9852
TRAIN: BATCH 0020/0157 | LOSS: 